In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

## 3.1 Simple forecasting methods

### Average Method:

The forecasts of all future values are equal to the average of the historical data:


$\hat{y}_{T+h|T}=\bar{y}=(y_{1}+..+y_{T})/T$

In [4]:
import datetime

In [22]:
def make_ts(start_date, periods, freq):
    y = pd.Series(pd.date_range(start_date, periods = periods, freq= freq))
    z = pd.Series(np.random.random((ts_df.shape[0],))*100)
    y = pd.concat([y, z], axis = 1)
    y.columns = ['Year', 'Observation']
    return y

In [23]:
start_date = '2009'
periods = 10
frequency = 'YS'

ts_df = make_ts(start_date, periods, frequency)

In [31]:
import sys
sys.path.append("..")

from src.helper import make_ts

In [32]:
ts_df = make_ts(2000,20)

NameError: name 'ts_df' is not defined